### 1. Initialize

In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import col, year, month
from sklearn.linear_model import LinearRegression

pars = {
    "account": "hdb90888",
    "user": "cristiscu",
    "password": "...",              # <-- do not hard-code this!!!
    "warehouse": "COMPUTE_WH",
    "role": "ACCOUNTADMIN",
    "database": "FINANCIAL_REGRESSION",
    "schema": "PUBLIC"
}
session = Session.builder.configs(pars).create()

# test connection
session.sql("select current_account() acct,"
    + " current_warehouse() wh,"
    + " current_database() db,"
    + " current_schema() schema,"
    + " current_version() version").show()

-----------------------------------------------------------------------
|"ACCT"    |"WH"        |"DB"                  |"SCHEMA"  |"VERSION"  |
-----------------------------------------------------------------------
|HDB90888  |COMPUTE_WH  |FINANCIAL_REGRESSION  |PUBLIC    |7.44.2     |
-----------------------------------------------------------------------



### 2. Query the data

In [2]:
table = "FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES"

# What financial data is available as a time-series from FRED?
session.sql(f"SELECT DISTINCT variable_name FROM {table}").show()

# What is the size of all the time-series data?
session.sql(f"SELECT COUNT(*) FROM {table}").show()

------------------------------------------------------
|"VARIABLE_NAME"                                     |
------------------------------------------------------
|Assets: Other Factors Supplying Reserve Balance...  |
|Other Deposits, Large Domestically Chartered Co...  |
|Liabilities and Capital: Liabilities: Deposits:...  |
|Securities in Bank Credit, Domestically Charter...  |
|Advance Retail Sales: Food Services and Drinkin...  |
|Assets: Securities Held Outright: Federal Agenc...  |
|Treasury and Agency Securities, Foreign-Related...  |
|Assets: Liquidity and Credit Facilities: Loans:...  |
|Liabilities and Capital: Capital: Surplus: Wedn...  |
|Liabilities and Capital: Capital: Surplus: Wedn...  |
------------------------------------------------------

--------------
|"COUNT(*)"  |
--------------
|2281559     |
--------------



### 3. Train a Linear Regression model

In [3]:
df = (session
    .table("FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES")
    .filter(col('VARIABLE_NAME') == 'Personal Consumption Expenditures: Chain-type Price Index, Seasonally adjusted, Monthly, Index 2017=100')
    .filter(col('DATE') >= '1972-01-01')
    .filter(month(col('DATE')) == 1)
    .select(year(col("DATE")).alias('"Year"'), col('VALUE').alias('PCE'))
    .orderBy(col('DATE'))
    .to_pandas())
print(df.tail())

x = df["Year"].to_numpy().reshape(-1,1)
y = df["PCE"].to_numpy()
model = LinearRegression().fit(x, y)

pred = model.predict([[2024]])
print(f'\nPrediction for 2024: {str(round(pred[0], 2))}')

    Year      PCE
47  2019  102.714
48  2020  104.458
49  2021  106.145
50  2022  112.829
51  2023  119.011

Prediction for 2024: 114.7


### 4. Create a UDF to do the scoring

In [4]:
def predict_pce(predYear: int) -> float:
    return model.predict([[predYear]])[0].round(2).astype(float)

session.udf.register(
    predict_pce, name="predict_pce_udf", 
    input_type=IntegerType(), return_type=FloatType(),
    stage_location="@stage", packages=["pandas", "scikit-learn"],
    is_permanent=True, replace=True)

session.sql("select predict_pce_udf(2024)").show()

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


---------------------------
|"PREDICT_PCE_UDF(2024)"  |
---------------------------
|114.7                    |
---------------------------

